In [1]:
import numpy as np
import pandas as pd

In [2]:
books = pd.read_csv('Books.csv')
ratings = pd.read_csv('Ratings.csv')
users = pd.read_csv('Users.csv')

/tmp/ipykernel_6958/378180666.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books = pd.read_csv('Books.csv')


In [3]:
books.sample()

,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
196874,0525933662,The Judas Kiss,Norman Katkov,1991,Penguin USA,http://images.amazon.com/images/P/0525933662.0...,http://images.amazon.com/images/P/0525933662.0...,http://images.amazon.com/images/P/0525933662.0...


In [4]:
ratings.sample()

,User-ID,ISBN,Book-Rating
660643,160408,3442545668,0


In [5]:
users.sample()

,User-ID,Location,Age
88000,88001,"duluth, georgia, usa",47.0


In [6]:
books_rating = books.merge(ratings,on='ISBN')

In [7]:
n_books =(books_rating.groupby('Book-Title').count()['Book-Rating']).reset_index()

In [8]:
n_books = n_books[n_books['Book-Rating']>150]

## Extracting Users with minimum 50 rated books

In [9]:
users_rating = users.merge(books_rating,on='User-ID')

In [10]:
n_users = users_rating.groupby('User-ID').count()['Book-Rating'].reset_index()

In [11]:
n_users = n_users[n_users['Book-Rating']>200]

In [12]:
sel_users = users_rating[users_rating['User-ID'].isin(n_users['User-ID'].values)]

In [13]:
data = sel_users[sel_users['Book-Title'].isin(n_books['Book-Title'].values)]

In [14]:
data[data['Book-Title']=='1984']

,User-ID,Location,Age,ISBN,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L,Book-Rating
383,254,"minneapolis, minnesota, usa",24.0,0451524934,1984,George Orwell,1990,Signet Book,http://images.amazon.com/images/P/0451524934.0...,http://images.amazon.com/images/P/0451524934.0...,http://images.amazon.com/images/P/0451524934.0...,9
18977,7346,"sunnyvale, california, usa",49.0,0451519841,1984,George Orwell,1980,New Amer Library,http://images.amazon.com/images/P/0451519841.0...,http://images.amazon.com/images/P/0451519841.0...,http://images.amazon.com/images/P/0451519841.0...,8
32033,11676,"n/a, n/a, n/a",NaN,0451524934,1984,George Orwell,1990,Signet Book,http://images.amazon.com/images/P/0451524934.0...,http://images.amazon.com/images/P/0451524934.0...,http://images.amazon.com/images/P/0451524934.0...,0
33317,11676,"n/a, n/a, n/a",NaN,0451519841,1984,George Orwell,1980,New Amer Library,http://images.amazon.com/images/P/0451519841.0...,http://images.amazon.com/images/P/0451519841.0...,http://images.amazon.com/images/P/0451519841.0...,0
34022,11676,"n/a, n/a, n/a",NaN,0452262933,1984,George Orwell,1983,Plume Books,http://images.amazon.com/images/P/0452262933.0...,http://images.amazon.com/images/P/0452262933.0...,http://images.amazon.com/images/P/0452262933.0...,10
...,...,...,...,...,...,...,...,...,...,...,...,...
961381,258534,"mcqueeney, texas, usa",32.0,0451524934,1984,George Orwell,1990,Signet Book,http://images.amazon.com/images/P/0451524934.0...,http://images.amazon.com/images/P/0451524934.0...,http://images.amazon.com/images/P/0451524934.0...,0
968104,260897,"chicago, illinois, usa",25.0,0452262933,1984,George Orwell,1983,Plume Books,http://images.amazon.com/images/P/0452262933.0...,http://images.amazon.com/images/P/0452262933.0...,http://images.amazon.com/images/P/0452262933.0...,0
989095,266753,"oakland, california, usa",NaN,0451524934,1984,George Orwell,1990,Signet Book,http://images.amazon.com/images/P/0451524934.0...,http://images.amazon.com/images/P/0451524934.0...,http://images.amazon.com/images/P/0451524934.0...,0
1008034,271705,"atlanta, georgia, usa",25.0,0451524934,1984,George Orwell,1990,Signet Book,http://images.amazon.com/images/P/0451524934.0...,http://images.amazon.com/images/P/0451524934.0...,http://images.amazon.com/images/P/0451524934.0...,10


In [15]:
pt = data.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')

In [16]:
pt = pt.fillna(0)

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
recoms_df = cosine_similarity(pt)

In [19]:
index = np.where(pt.index=='1984')[0][0]

In [20]:
sorted(list(enumerate(recoms_df[0])),key=lambda x: x[1],reverse=True)[1:6]

[(38, 0.2702651417103732),
 (364, 0.26396193711234966),
 (291, 0.23967698903671683),
 (63, 0.2366937434740099),
 (435, 0.23299389358170397)]

In [21]:
num = list(enumerate(recoms_df[0]))

In [37]:
data = []
def recommend(movie_name):
    index = np.where(pt.index==movie_name)[0][0]
    movies_ind = sorted(list(enumerate(recoms_df[index])),key=lambda x: x[1],reverse=True)[1:6]
    for i in movies_ind:
        
        item = []
        temp_df = books[books['Book-Title'] == pt.index[i[0]]]
        item.extend(temp_df.drop_duplicates('Book-Title')['Book-Title'].values)
        item.extend(temp_df.drop_duplicates('Book-Title')['Book-Author'].values)
        item.extend(temp_df.drop_duplicates('Book-Title')['Image-URL-M'].values)
        data.append(item)
        
    print(data)        

In [38]:
recommend('1984')

[['Animal Farm', 'George Orwell', 'http://images.amazon.com/images/P/0451526341.01.MZZZZZZZ.jpg'], ["The Handmaid's Tale", 'Margaret Atwood', 'http://images.amazon.com/images/P/0449212602.01.MZZZZZZZ.jpg'], ["Slaughterhouse Five or the Children's Crusade: A Duty Dance With Death", 'Kurt Vonnegut', 'http://images.amazon.com/images/P/0440180295.01.MZZZZZZZ.jpg'], ['Brave New World', 'Aldous Huxley', 'http://images.amazon.com/images/P/0060809833.01.MZZZZZZZ.jpg'], ['The Vampire Lestat (Vampire Chronicles, Book II)', 'ANNE RICE', 'http://images.amazon.com/images/P/0345313860.01.MZZZZZZZ.jpg']]


In [35]:
import pickle

In [41]:
pickle.dump(pt,open('pt.pkl','wb'))
pickle.dump(books,open('books.pkl','wb'))
pickle.dump(recoms_df,open('recoms.pkl','wb'))